In [1]:
import requests
import random
import pandas as pd
from bs4 import BeautifulSoup
from headers import headers_list
from loading_bar import log_progress

In [2]:
url = 'https://graduan.com/accenture'
url

'https://graduan.com/accenture'

In [42]:
def get_company_info(url):
    company = {}
    page = requests.get(url, headers=random.choice(headers_list))
    soup = BeautifulSoup(page.content, 'html.parser')
    info = soup.find('div', class_='small-section')
    for div in info.findAll('div'):
        head = div.find('h4')
        if head is not None:
            a = div.find('a')
            if a is not None:
                body = a['href'].strip('\n')
            else:
                body = div.text.replace(head.text, '').strip('\n')
            company[head.text] = body
    return company

In [43]:
get_company_info(url)

{'Contact Number': 'tel:03 2088 4000',
 'Fax No.': '03 20887000',
 'Email': '/cdn-cgi/l/email-protection#5e192c312909372a360b2d1e3f3d3d3b302a2b2c3b703d3133',
 'Address': 'Level 30 Menara Exchange 106, Lingkaran TRX Tun Razak Exchange, 55188 Kuala Lumpur.'}

In [15]:
def decode_email(e):
    de = ""
    k = int(e[:2], 16)
    for i in range(2, len(e)-1, 2):
        de += chr(int(e[i:i+2], 16)^k)
    return de

In [29]:
decode_email('a1e6d3ced6f6c8d5c9f4d2e1c0c2c2c4cfd5d4d3c48fc2cecc')

'GrowWithUs@accenture.com'